In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Face detection Surya/

/content/drive/MyDrive/Face detection Surya


In [ ]:
import os
import time
import uuid
import cv2

In [ ]:
import tensorflow as tf
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_image(x):
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [ ]:
train_images = tf.data.Dataset.list_files('/content/drive/MyDrive/Face detection Surya/aug_data/train/images/*.jpg',shuffle=False)
train_images =train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x , (120,120)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
test_images = tf.data.Dataset.list_files('/content/drive/MyDrive/Face detection Surya/aug_data/test/images/*.jpg',shuffle=False)
test_images =test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x , (120,120)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files('/content/drive/MyDrive/Face detection Surya/aug_data/val/images/*.jpg',shuffle=False)
val_images =val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x , (120,120)))
val_images = val_images.map(lambda x: x/255)

In [ ]:
train_images.as_numpy_iterator().next()

array([[[0.54509807, 0.63143384, 0.70625   ],
        [0.5436887 , 0.6250613 , 0.6943015 ],
        [0.5643995 , 0.63890934, 0.700674  ],
        ...,
        [0.04601716, 0.090625  , 0.10091912],
        [0.04852941, 0.11470588, 0.12696078],
        [0.0432598 , 0.12120098, 0.13259804]],

       [[0.5362745 , 0.6186274 , 0.70098037],
        [0.5376226 , 0.61764705, 0.6917892 ],
        [0.5459559 , 0.6204657 , 0.68713236],
        ...,
        [0.0435049 , 0.07977941, 0.10575981],
        [0.0713848 , 0.12328431, 0.14620098],
        [0.08541667, 0.15275735, 0.16452205]],

       [[0.55110294, 0.62169117, 0.7007353 ],
        [0.57757354, 0.64816177, 0.72659314],
        [0.5583946 , 0.62898284, 0.7074142 ],
        ...,
        [0.07273284, 0.11672794, 0.12414216],
        [0.05790441, 0.1353554 , 0.13192402],
        [0.03719363, 0.11954657, 0.12346814]],

       ...,

       [[0.7261642 , 0.79381126, 0.8423407 ],
        [0.73719364, 0.7920956 , 0.83523285],
        [0.7333946 , 0

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(),'r' , encoding="utf-8") as f:
        label = json.load(f)

    return [label['class']] , label['bbox']

In [ ]:
train_labels = tf.data.Dataset.list_files('/content/drive/MyDrive/Face detection Surya/aug_data/train/labels/*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels , [x] ,[tf.uint8 , tf.float16]))


In [ ]:
test_labels = tf.data.Dataset.list_files('/content/drive/MyDrive/Face detection Surya/aug_data/test/labels/*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels , [x] ,[tf.uint8 , tf.float16]))


In [ ]:
val_labels = tf.data.Dataset.list_files('/content/drive/MyDrive/Face detection Surya/aug_data/val/labels/*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels , [x] ,[tf.uint8 , tf.float16]))


In [ ]:
len(train_images) , len(train_labels) , len(test_images) , len(test_labels) , len(val_images) , len(val_labels)

(3840, 3840, 840, 844, 780, 780)

In [ ]:
train = tf.data.Dataset.zip((train_images , train_labels))
train = train.shuffle(5000)
train = train.batch(8)
train= train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images , test_labels))
test = test.shuffle(1300)
test = test.batch(8)
test= test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images , val_labels))
val = val.shuffle(1000)
val = val.batch(8)
val= val.prefetch(4)

In [ ]:
train.as_numpy_iterator().next()[0].shape , train.as_numpy_iterator().next()

((8, 120, 120, 3),
 (array([[[[4.47855383e-01, 4.19914216e-01, 4.22487736e-01],
           [4.55147058e-01, 4.16299015e-01, 4.29840684e-01],
           [4.85661775e-01, 4.47426468e-01, 4.60661769e-01],
           ...,
           [5.82536757e-01, 5.74693620e-01, 5.78615189e-01],
           [5.83517134e-01, 5.71384788e-01, 5.75306356e-01],
           [5.77389717e-01, 5.69240212e-01, 5.68566203e-01]],
  
          [[4.88848031e-01, 4.45710778e-01, 4.61397052e-01],
           [4.68443632e-01, 4.25428927e-01, 4.41053927e-01],
           [4.69914228e-01, 4.33762252e-01, 4.45955873e-01],
           ...,
           [5.65196097e-01, 5.58333337e-01, 5.61764717e-01],
           [5.74632347e-01, 5.67769587e-01, 5.71200967e-01],
           [5.59129894e-01, 5.51654398e-01, 5.55392146e-01]],
  
          [[4.70772058e-01, 4.46691185e-01, 4.54718143e-01],
           [4.67953444e-01, 4.44424033e-01, 4.44424033e-01],
           [4.59007353e-01, 4.43198532e-01, 4.39338237e-01],
           ...,
          

In [ ]:
train.as_numpy_iterator().next()[1]

(array([[0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0]], dtype=uint8),
 array([[0.    , 0.    , 0.    , 0.    ],
        [0.4998, 0.262 , 0.9194, 0.871 ],
        [0.137 , 0.2411, 0.412 , 0.637 ],
        [0.555 , 0.2026, 0.946 , 0.6445],
        [0.2494, 0.336 , 0.6753, 0.805 ],
        [0.627 , 0.832 , 0.9194, 1.    ],
        [0.    , 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , 0.    ]], dtype=float16))

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

In [ ]:
model = VGG19(include_top=False , input_tensor=None, input_shape=(128 , 128 , 3))


80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , Conv2D , Dense , GlobalMaxPooling2D

In [ ]:
def build_model():
    input_shape= Input(shape=(120 ,120 ,3))

    vgg = VGG19(include_top=False)(input_shape)

    func1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048 , activation='relu')(func1)
    class2 = Dense(1 , activation='sigmoid')(class1)

    func2 = GlobalMaxPooling2D()(vgg)
    reg1 = Dense(2048 , activation='relu')(func2)
    reg2 = Dense(4 , activation='sigmoid')(reg1)

    facetracker = Model(inputs=input_shape , outputs=[class2 ,reg2])
    return facetracker

In [ ]:
facetracker = build_model()

In [ ]:
facetracker.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 120, 120, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg19 (Functional)             (None, None, None,   20024384    ['input_2[0][0]']                
                                512)                                                              
                                                                                                  
 global_max_pooling2d (GlobalMa  (None, 512)         0           ['vgg19[0][0]']                  
 xPooling2D)                                                                                  

In [ ]:
X , Y = train.as_numpy_iterator().next()

In [ ]:
X.shape , X

((8, 120, 120, 3),
 array([[[[0.9279412 , 0.87414217, 0.8726103 ],
          [0.921201  , 0.87708336, 0.8805147 ],
          [0.92677695, 0.89050245, 0.89001226],
          ...,
          [0.7393995 , 0.7286152 , 0.7325368 ],
          [0.72996324, 0.7148897 , 0.7119485 ],
          [0.76599264, 0.7405025 , 0.7287378 ]],
 
         [[0.9184436 , 0.8586397 , 0.86090684],
          [0.91311276, 0.88664216, 0.86311275],
          [0.89595586, 0.8856618 , 0.8567402 ],
          ...,
          [0.76960784, 0.7343137 , 0.74607843],
          [0.7622549 , 0.74558824, 0.7439951 ],
          [0.759375  , 0.74669117, 0.7298407 ]],
 
         [[0.9520221 , 0.8814951 , 0.88210785],
          [0.9322304 , 0.8979167 , 0.8699755 ],
          [0.92015934, 0.90447307, 0.8613358 ],
          ...,
          [0.7584559 , 0.7357843 , 0.7431985 ],
          [0.7378064 , 0.7359681 , 0.7368873 ],
          [0.74699754, 0.7471201 , 0.73995095]],
 
         ...,
 
         [[0.5846814 , 0.5805147 , 0.5773897 ],

In [ ]:
Y

(array([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1]], dtype=uint8),
 array([[0.3535 , 0.2163 , 0.724  , 0.7397 ],
        [0.     , 0.1312 , 0.4294 , 0.578  ],
        [0.8604 , 0.2267 , 1.     , 0.6934 ],
        [0.5454 , 0.3914 , 0.91   , 0.825  ],
        [0.5195 , 0.3787 , 0.886  , 0.7925 ],
        [0.07794, 0.2783 , 0.466  , 0.6885 ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.8247 , 0.2134 , 1.     , 0.68   ]], dtype=float16))

In [ ]:
classes , coords = facetracker.predict(X)
classes , coords

1/1 [==============================] - 8s 8s/step


(array([[0.50901777],
        [0.7054194 ],
        [0.5911368 ],
        [0.53355443],
        [0.52537864],
        [0.5621796 ],
        [0.5465333 ],
        [0.58284944]], dtype=float32),
 array([[0.3649418 , 0.47770125, 0.48471436, 0.45149362],
        [0.2972489 , 0.57707447, 0.44534183, 0.44810146],
        [0.34366181, 0.50530994, 0.35540405, 0.47901607],
        [0.35463557, 0.40832987, 0.4834462 , 0.53114134],
        [0.30067524, 0.43324584, 0.5178823 , 0.47695535],
        [0.27640054, 0.46192908, 0.51609373, 0.53226465],
        [0.4135725 , 0.48434347, 0.4700149 , 0.5105124 ],
        [0.33621922, 0.53396696, 0.34703156, 0.4781466 ]], dtype=float32))

In [ ]:
len(train)

480

In [ ]:
lr_decay = (1./0.75 -1)/len(train)

In [ ]:
opt=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001 , decay=lr_decay)

In [ ]:
def localization_loss(y_true , ypred):
    coord = tf.reduce_sum(tf.square(y_true[: , :2] - ypred[: , :2]))

    h_true = y_true[: , 3] - y_true[: ,1]
    w_true = y_true[:,2] - y_true[: , 0]

    h_pred= ypred[: ,3] - ypred[: , 1]
    w_pred = ypred[: ,2] - ypred[: , 0]

    size = tf.reduce_sum(tf.square(w_true - w_pred)+ tf.square(h_true - h_pred))

    return coord + size



In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy()
regressloss = localization_loss


In [ ]:
localization_loss(Y[1] , coords).numpy()

3.3963656

In [ ]:
classloss(Y[0] , classes)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5910456>

In [ ]:
class FaceTracker(Model):
    def __init__(self , eyetracker , **kwargs):
        super().__init__(**kwargs)
        self.model = facetracker

    def compile(self , opt , classloss , localizationloss , **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt

    def train_step(self , batch , ** kwargs):

        X ,y = batch

        with tf.GradientTape() as tape:
            classes , coords = self.model(X , training=True)

            batch_classloss = self.closs(y[0] , classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32),coords)

            total_loss = batch_localizationloss + 0.5*batch_classloss

            grad = tape.gradient(total_loss , self.model.trainable_variables)

        opt.apply_gradients(zip(grad , self.model.trainable_variables))

        return {"total loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}

    def test_step(self , batch , **kwargs):
        X , y = batch

        classes , coords = self.model(X , training=False)

        batch_classloss = self.closs(y[0] , classes)
        batch_regressloss = self.lloss(tf.cast(y[1], tf.float32),coords)
        total_loss = batch_regressloss+ 0.5*batch_classloss

        return {"total loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_regressloss}

    def call(self , X , **kwargs):
        return self.model(X , **kwargs)


In [ ]:
model = FaceTracker(facetracker)

In [ ]:
model.compile(opt , classloss , regressloss)

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train,epochs = 50 , validation_data=val,
                 callbacks=[tensorboard_callback])

Epoch 1/50
480/480 [==============================] - 275s 535ms/step - total loss: 0.6359 - class_loss: 0.1966 - regress_loss: 0.5376 - val_total loss: 0.1677 - val_class_loss: 0.0741 - val_regress_loss: 0.1307
Epoch 2/50
480/480 [==============================] - 55s 85ms/step - total loss: 0.1497 - class_loss: 0.0529 - regress_loss: 0.1232 - val_total loss: 1.0792 - val_class_loss: 0.8143 - val_regress_loss: 0.6720
Epoch 3/50
480/480 [==============================] - 55s 85ms/step - total loss: 0.1000 - class_loss: 0.0317 - regress_loss: 0.0842 - val_total loss: 0.9674 - val_class_loss: 0.7862 - val_regress_loss: 0.5743
Epoch 4/50
480/480 [==============================] - 55s 84ms/step - total loss: 0.0680 - class_loss: 0.0205 - regress_loss: 0.0577 - val_total loss: 0.0057 - val_class_loss: 5.9042e-04 - val_regress_loss: 0.0054
Epoch 5/50
480/480 [==============================] - 55s 85ms/step - total loss: 0.0425 - class_loss: 0.0150 - regress_loss: 0.0350 - val_total loss: 0.4

In [ ]:
hist.history

{'total loss': [0.164281964302063,
  0.10173428803682327,
  0.021811295300722122,
  0.003010910004377365,
  0.010016990825533867,
  0.007830968126654625,
  0.004855073522776365,
  0.001695781946182251,
  0.003960965201258659,
  0.004113123286515474,
  0.001883433316834271,
  0.009293238632380962,
  0.0036613408010452986,
  0.002678768942132592,
  0.004270331002771854,
  0.0024616592563688755,
  0.001995797734707594,
  0.002763368422165513,
  0.003980572801083326,
  0.0017397520132362843,
  0.0012990301474928856,
  0.0004244384472258389,
  0.000468916492536664,
  0.00023782857169862837,
  0.005545103922486305,
  0.0006227773847058415,
  0.0008291176636703312,
  0.0006046078633517027,
  0.0004938017809763551,
  0.000847482355311513,
  0.00031734260846860707,
  0.0003220132493879646,
  0.0006523724878206849,
  0.0007499392377212644,
  0.0008366898400709033,
  0.0010876497253775597,
  0.0003873057139571756,
  0.0011624984908849,
  0.0019268706673756242,
  0.0008293162100017071,
  0.0002336

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
facetracker.save('facetrackerSurya50.h5')